In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Input, Flatten, BatchNormalization
from sklearn.model_selection import train_test_split

In [2]:
import time, datetime
from fyers_apiv3 import fyersModel
import authenticator
client_id = "ZHQ4IJL7TI-100"
access_token = authenticator.authenticate()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")
market_status = fyers.market_status()

"Market: " + market_status["marketStatus"][1]['status']

'Market: CLOSED'

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA
from indicators.candle import OHLC
from indicators import epoch_to_time

In [5]:
date_yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_yesterday, date_100_p

('2025-02-18', '2024-11-11')

In [6]:
response = fyers.history(data={
                                "symbol": "NSE:HDFCBANK-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_yesterday,
                                "cont_flag": "1"
                                })

len(response['candles'])

26625

In [7]:
date_temp = []
final_candles_list = []
for i in response['candles']:
    if i[0] not in date_temp:
        final_candles_list.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list)

26250

In [8]:
candles = np.array(final_candles_list)
candles.shape

(26250, 6)

In [9]:
sync = SyncInd(
    Alligator(show_jaw=False, show_teeth=False),
    KAMA(highlight=True),
    SMA(5),
)

for c in candles:
    sync.append(c)
sync.data().shape

(26250, 10)

In [10]:
X, y = np.array(sync.data()[:, 1:9], np.float32), np.array(sync.data()[:, 9], np.float32)
X.shape, y.shape

((26250, 8), (26250,))

In [11]:
train_thres = 100
x_train, x_test = X[:-train_thres], X[-train_thres:]
y_train, y_test = y[:-train_thres], y[-train_thres:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((26150, 8), (26150,), (100, 8), (100,))

In [19]:
# Assuming data is already loaded
num_features = 8  # Since the shape is (26250, 10)

# Define the model
def build_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(num_features,)),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(1)  # Output layer (adjust activation for classification)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss='mae',  # Use 'categorical_crossentropy' for classification
                  metrics=['mae'])
    return model

model = build_model()
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,249 (270.50 KB)

 Trainable params: 68,225 (266.50 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [20]:
# Train the model
model.fit(tf.expand_dims(x_train, axis=-1), y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1000us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
601/654 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - loss: nan - mae: nan

KeyboardInterrupt: 

In [ ]:
evaluation = model.evaluate(tf.expand_dims(x_test, axis=-1), y_test)
print(f"Test Loss: {evaluation[0]}, Test MAE: {evaluation[1]}")

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming data is already loaded
num_features = 8  # Updated to match the dataset shape

# Define the model
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(num_features,)),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.Dense(1)  # Output layer for regression
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), 
                  loss='mse',  # Use 'categorical_crossentropy' for classification
                  metrics=['mae'])
    return model

# Create and summarize the model
model = build_model()
model.summary()


# Normalize the dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

# Remove NaN and Inf values from y_train
y_train = np.nan_to_num(y_train, nan=0.0, posinf=1.0, neginf=-1.0)

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Normalize test data
x_test = scaler.transform(x_test)
y_test = np.nan_to_num(y_test, nan=0.0, posinf=1.0, neginf=-1.0)

evaluation = model.evaluate(x_test, y_test)
print(f"Test Loss: {evaluation[0]}, Test MAE: {evaluation[1]}")



/Users/anodic_passion/PycharmProjects/108decillion/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,249 (270.50 KB)

 Trainable params: 68,225 (266.50 KB)

 Non-trainable params: 1,024 (4.00 KB)

Epoch 1/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 1m

In [14]:
print("Model Prediction", model.predict(x_test[-3:])), y_test[-3:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Model Prediction [[nan]
 [nan]
 [nan]]


(None, array([1722.  , 1722.12, 1722.27], dtype=float32))